In [2]:
import warnings
import subprocess
from pathlib import Path

import nglview as nv
from openbabel import pybel
from opencadd.structure.core import Structure

import pandas as pd


In [3]:
warnings.filterwarnings("ignore")
ob_log_handler = pybel.ob.OBMessageHandler()
pybel.ob.obErrorLog.SetOutputLevel(0)

In [4]:
com_data = pd.read_csv("./data/fak_pdbbind_dataset.csv", skiprows=1)

In [5]:
com_data

,ID,PDB code,Subset,Complex Type,Resolution,Affinity Data,pKd pKi pIC50,Release Year,Protein Name,Ligand Name,...,Exact Mass,No. of atoms,No. of bonds,Polar Surface Area,XLOGP3,open banel LogP,HB donor,HB acceptor,Rotatable bonds,Canonical SMILES
0,1,2etm,general,Protein-Ligand,2.30,IC50=0.212uM,6.67,2006,"Focal Adhesion Kinase Domain, FAK1",7PY,...,377.149,47,50,83.32,3.33,3.66,1,3,6,COc1cc(Nc2ncc3c(n2)n(cc3)c2ccccn2)cc(c1OC)OC
1,2,2jkk,general,Protein-Ligand,2.00,IC50=6.24nM,8.20,2008,FAK,BI9,...,468.168,58,61,100.64,4.13,4.42,3,3,8,CNC(=O)c1ccccc1Nc1nc(ncc1Cl)Nc1ccc(cc1OC)N1CCOCC1
2,3,2jkm,general,Protein-Ligand,2.31,IC50=17.76nM,7.75,2008,FAK,BII,...,546.205,70,73,133.47,4.76,4.77,4,4,10,C[NH2+][C@@H]1CCN(C1)c1ccc(c(c1)OC)Nc1ncc(c(n1...
3,4,2jko,general,Protein-Ligand,1.65,IC50=7.7nM,8.11,2008,FAK,BIJ,...,494.207,64,68,87.06,3.52,4.26,3,3,6,COc1cc(ccc1Nc1ncc(c(n1)Nc1cccc2c1C(=O)N(C2)C)C...
4,5,2jkq,general,Protein-Ligand,2.60,IC50=7.7nM,8.11,2008,FAK,VG8,...,562.270,77,82,87.06,5.70,5.97,3,3,7,COc1ccccc1Nc1ncc(c(n1)Nc1ccc(c2c1C(=O)N(C2)C)N...
5,6,3bz3,general,Protein-Ligand,2.20,IC50=1.5nM,8.82,2008,FAK,YAM,...,507.130,55,58,137.59,2.04,4.50,3,6,7,O=C1Nc2c(C1)cc(cc2)Nc1ncc(c(n1)NCc1cccnc1N(S(=...
6,7,4q9s,general,Protein-Ligand,2.07,IC50=156uM,3.81,2014,Focal adhesion kinase 1 FAK,30G,...,203.069,24,26,53.93,0.45,0.16,1,1,0,O=C1NN=C2N(C1)c1ccccc1OC2


In [6]:
import requests
from opencadd.structure.core import Structure

def fetch_pdb_data(pdb_id):
    url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
    response = requests.get(url)
    response.raise_for_status()
    with open(f"{pdb_id}.pdb", 'w') as file:
        file.write(response.text)
    return f"{pdb_id}.pdb"

structure_list = []
for i in com_data["PDB code"]:
    pdb_file = fetch_pdb_data(i)
    structure = Structure(pdb_file)
    structure_list.append(structure)

print(structure_list)


[<Universe with 4400 atoms>, <Universe with 2252 atoms>, <Universe with 2198 atoms>, <Universe with 2332 atoms>, <Universe with 2128 atoms>, <Universe with 2411 atoms>, <Universe with 2187 atoms>]


In [11]:
#for i in range(len(structure_list)):
#    protein = structure_list[i].select_atoms("protein")
#    protein.write(f"protein_{i}")

In [ ]:
#def pdb_to_pdbqt(pdb_path, pdbqt_path, pH=7.4):
    """
    Convert a PDB file to a PDBQT file needed by docking programs of the AutoDock family.

    Parameters
    ----------
    pdb_path: str or pathlib.Path
        Path to input PDB file.
    pdbqt_path: str or pathlib.path
        Path to output PDBQT file.
    pH: float
        Protonation at given pH.
    """
    molecule = list(pybel.readfile("pdb", str(pdb_path)))[0]
    # add hydrogens at given pH
    molecule.OBMol.CorrectForPH(pH)
    molecule.addh()
    # add partial charges to each atom
    for atom in molecule.atoms:
        atom.OBAtom.GetPartialCharge()
    molecule.write("pdbqt", str(pdbqt_path), overwrite=True)
    return

In [ ]:
# Convert protein to PDBQT format
for i in range(len(structure_list)):
    pdb_to_pdbqt(f"protein_{i}.pdb", f"protein_{i}.pdbqt")
    

In [12]:
# Zn27 smiles
smiles = "COc1ccc([C@@H]2CCCN2C(=O)Nc2cc(C(F)(F)F)ccc2N2CCOCC2)cc1"

In [ ]:
def smiles_to_pdbqt(smiles, pdbqt_path, pH=7.4):
    """
    Convert a SMILES string to a PDBQT file needed by docking programs of the AutoDock family.

    Parameters
    ----------
    smiles: str
        SMILES string.
    pdbqt_path: str or pathlib.path
        Path to output PDBQT file.
    pH: float
        Protonation at given pH.
    """
    molecule = pybel.readstring("smi", smiles)
    # add hydrogens at given pH
    molecule.OBMol.CorrectForPH(pH)
    molecule.addh()
    # generate 3D coordinates
    molecule.make3D(forcefield="mmff94s", steps=10000)
    # add partial charges to each atom
    for atom in molecule.atoms:
        atom.OBAtom.GetPartialCharge()
    molecule.write("pdbqt", str(pdbqt_path), overwrite=True)
    return

In [ ]:
smiles_to_pdbqt(smiles, "ligand_Zn27.pdbqt")

In [7]:
pocketsize_list = []
pocketcenter_list = []
for i in range(len(com_data)):
    ligand_resname = com_data["Ligand Name"][i]
    ligand = structure_list[i].select_atoms(f"resname {ligand_resname}")
    pocket_center = (ligand.positions.max(axis=0) + ligand.positions.min(axis=0)) / 2
    pocket_size = ligand.positions.max(axis=0) - ligand.positions.min(axis=0) +5
    pocketsize_list.append(pocket_size)
    pocketcenter_list.append(pocket_center)

In [8]:
pocketcenter_list

[array([-5.1585, 10.445 , 17.7845], dtype=float32),
 array([ 8.6615   ,  1.2759999, 27.473    ], dtype=float32),
 array([ 8.4925   ,  1.4609998, 27.6255   ], dtype=float32),
 array([29.143   , 23.544498, 35.716   ], dtype=float32),
 array([11.1735    ,  0.70149994, 26.069     ], dtype=float32),
 array([9.8445, 2.763 , 5.109 ], dtype=float32),
 array([ 6.2605,  2.9775, 25.096 ], dtype=float32)]

In [9]:
pocketsize_list

[array([22.509   , 12.940001, 32.963   ], dtype=float32),
 array([12.743, 14.69 , 16.098], dtype=float32),
 array([13.424999 , 14.992    , 15.8029995], dtype=float32),
 array([13.327999 , 14.6970005, 17.428    ], dtype=float32),
 array([18.591, 16.111, 14.236], dtype=float32),
 array([13.869, 14.53 , 15.858], dtype=float32),
 array([12.424999, 11.503   ,  9.216   ], dtype=float32)]

In [10]:
pocketcenter_list = pd.Series(pocketsize_list)
pocketsize_list = pd.Series(pocketsize_list)

In [11]:
pocketinfo = pd.concat([pocketcenter_list, pocketsize_list], axis=1)
pocketinfo

,0,1
0,"[22.509, 12.940001, 32.963]","[22.509, 12.940001, 32.963]"
1,"[12.743, 14.69, 16.098]","[12.743, 14.69, 16.098]"
2,"[13.424999, 14.992, 15.8029995]","[13.424999, 14.992, 15.8029995]"
3,"[13.327999, 14.6970005, 17.428]","[13.327999, 14.6970005, 17.428]"
4,"[18.591, 16.111, 14.236]","[18.591, 16.111, 14.236]"
5,"[13.869, 14.53, 15.858]","[13.869, 14.53, 15.858]"
6,"[12.424999, 11.503, 9.216]","[12.424999, 11.503, 9.216]"


In [12]:
ligandname = []
for i in com_data["Ligand Name"]:
    ligandname.append(i)

ligandname = pd.Series(ligandname)
ligandname

0    7PY
1    BI9
2    BII
3    BIJ
4    VG8
5    YAM
6    30G
dtype: object

In [13]:
pocketinfo = pd.concat([ligandname, pocketinfo], axis=1)
pocketinfo

,0,0,1
0,7PY,"[22.509, 12.940001, 32.963]","[22.509, 12.940001, 32.963]"
1,BI9,"[12.743, 14.69, 16.098]","[12.743, 14.69, 16.098]"
2,BII,"[13.424999, 14.992, 15.8029995]","[13.424999, 14.992, 15.8029995]"
3,BIJ,"[13.327999, 14.6970005, 17.428]","[13.327999, 14.6970005, 17.428]"
4,VG8,"[18.591, 16.111, 14.236]","[18.591, 16.111, 14.236]"
5,YAM,"[13.869, 14.53, 15.858]","[13.869, 14.53, 15.858]"
6,30G,"[12.424999, 11.503, 9.216]","[12.424999, 11.503, 9.216]"


In [14]:
pocketinfo.columns = ["Ligand Name", "pocket center", "pocket size"]
pocketinfo

,Ligand Name,pocket center,pocket size
0,7PY,"[22.509, 12.940001, 32.963]","[22.509, 12.940001, 32.963]"
1,BI9,"[12.743, 14.69, 16.098]","[12.743, 14.69, 16.098]"
2,BII,"[13.424999, 14.992, 15.8029995]","[13.424999, 14.992, 15.8029995]"
3,BIJ,"[13.327999, 14.6970005, 17.428]","[13.327999, 14.6970005, 17.428]"
4,VG8,"[18.591, 16.111, 14.236]","[18.591, 16.111, 14.236]"
5,YAM,"[13.869, 14.53, 15.858]","[13.869, 14.53, 15.858]"
6,30G,"[12.424999, 11.503, 9.216]","[12.424999, 11.503, 9.216]"


In [15]:
pocketinfo["pocket center"][0][0]

22.509

In [16]:
def run_smina(
    ligand_path, protein_path, out_path, pocket_center, pocket_size, num_poses=10, exhaustiveness=8
):
    """
    Perform docking with Smina.

    Parameters
    ----------
    ligand_path: str or pathlib.Path
        Path to ligand PDBQT file that should be docked.
    protein_path: str or pathlib.Path
        Path to protein PDBQT file that should be docked to.
    out_path: str or pathlib.Path
        Path to which docking poses should be saved, SDF or PDB format.
    pocket_center: iterable of float or int
        Coordinates defining the center of the binding site.
    pocket_size: iterable of float or int
        Lengths of edges defining the binding site.
    num_poses: int
        Maximum number of poses to generate.
    exhaustiveness: int
        Accuracy of docking calculations.

    Returns
    -------
    output_text: str
        The output of the Smina calculation.
    """
    output_text = subprocess.check_output(
        [
            # In Max   #########
            #"./smina.osx.12",

            # In Linux #########
            "./smina.linux",
            
            "--ligand",
            str(ligand_path),
            "--receptor",
            str(protein_path),
            "--out",
            str(out_path),
            "--center_x",
            str(pocket_center[0]),
            "--center_y",
            str(pocket_center[1]),
            "--center_z",
            str(pocket_center[2]),
            "--size_x",
            str(pocket_size[0]),
            "--size_y",
            str(pocket_size[1]),
            "--size_z",
            str(pocket_size[2]),
            "--num_modes",
            str(num_poses),
            "--exhaustiveness",
            str(exhaustiveness),
        ],
        universal_newlines=True,  # needed to capture output text
    )
    return output_text

In [17]:
Zn27docking = open("Zn27docking_score_twice.txt", mode="w")

In [18]:
for i in range(7):
    results = run_smina(
        "./data/pdbqt_file/ligand_Zn27.pdbqt",
        f"./data/pdbqt_file/protein_{i}.pdbqt",
        f"docking_poses_{i}.sdf",
        pocketinfo['pocket center'][i],
        pocketinfo['pocket size'][i]
    )
    Zn27docking.write(results)
    print(results)

Zn27docking.close()

   _______  _______ _________ _        _______ 
  (  ____ \(       )\__   __/( (    /|(  ___  )
  | (    \/| () () |   ) (   |  \  ( || (   ) |
  | (_____ | || || |   | |   |   \ | || (___) |
  (_____  )| |(_)| |   | |   | (\ \) ||  ___  |
        ) || |   | |   | |   | | \   || (   ) |
  /\____) || )   ( |___) (___| )  \  || )   ( |
  \_______)|/     \|\_______/|/    )_)|/     \|


smina is based off AutoDock Vina. Please cite appropriately.

Weights      Terms
-0.035579    gauss(o=0,_w=0.5,_c=8)
-0.005156    gauss(o=3,_w=2,_c=8)
0.840245     repulsion(o=0,_c=8)
-0.035069    hydrophobic(g=0.5,_b=1.5,_c=8)
-0.587439    non_dir_h_bond(g=-0.7,_b=0,_c=8)
1.923        num_tors_div

Using random seed: 1110901213

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------


In [ ]:
# 5번쨰 protein good

In [19]:
# 6번쨰 단백질 결과가 이상해서 다시 해본 것이며 동일하게ㅐ 이상하게 나옴